In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns



Duplicate key in file WindowsPath('c:/Users/USER/anaconda3/envs/python3.9.15/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 273 ('font.monospace:  DejaVu Sans Mono, Bitstream Vera Sans Mono, Computer Modern Typewriter, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace')


In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

# USER Tokenize

In [3]:
users_df.head(5)

,user_id,gender,occupation_titles,interests,recreation_names
0,54ccaa73a784960a00948687,female,NaN,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",NaN
1,54dca4456d7d350900e86bae,male,NaN,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",NaN
2,54e421bac5c9c00900cd8d47,female,NaN,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",NaN
3,54e961d4c5c9c00900cd8d84,other,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",NaN
4,54e9b744c5c9c00900cd8d8a,other,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影"


In [4]:
id2user_mapping = users_df["user_id"].to_dict()
user2id_mapping = {v : k for k, v in id2user_mapping.items()}
len(id2user_mapping)

130566

In [5]:
def clean_data(x):
    x = x.replace("其他", "")
    x = x.replace("other", "")
    return x

filledna = users_df.fillna('')
features = ['gender', 'occupation_titles', 'interests', 'recreation_names']
filledna = filledna[features]
for feature in features:
    filledna[feature] = filledna[feature].apply(clean_data)
    
filledna.head(5)

,gender,occupation_titles,interests,recreation_names
0,female,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",
1,male,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",
2,female,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",
3,,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",
4,,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影"


In [6]:
def create_soup(x):
    return x['gender']+ ' ' + x['occupation_titles'] + ' ' + x['interests'] + ' ' + x['recreation_names']

filledna['soup'] = filledna.apply(create_soup, axis=1)

In [7]:
filledna['soup'].head(5)

0    female  職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設...
1    male  設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,...
2    female  設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技...
3     金融業 投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投...
4     資訊科技,法律、社會及文化專業,非營利組織 程式_網頁前端,投資理財_理財,投資理財_投資...
Name: soup, dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(filledna['soup'])
# Compute the cosine similarity matrix
cosine_sim2 = linear_kernel(tfidf_matrix, tfidf_matrix)



# count = CountVectorizer()
# count_matrix = count.fit_transform(filledna['soup'])
# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [24]:
import pickle
tfidf_save = {"TfidfVectorizer": tfidf, "Tfidf_matrix": tfidf_matrix}
with open('user_embedding.pickle', 'wb') as f:
    pickle.dump(tfidf_save, f)

In [9]:
print(tfidf_matrix.shape)
print(tfidf.vocabulary_)

(10000, 149)
{'female': 0, '職場技能_創業': 84, '藝術_電腦繪圖': 106, '設計_介面設計': 113, '設計_動態設計': 114, '設計_平面設計': 115, '投資理財_投資觀念': 24, '行銷_數位行銷': 107, '藝術_角色設計': 105, '藝術_繪畫與插畫': 102, '職場技能_個人品牌經營': 83, 'male': 1, '設計_應用設計': 116, '程式_程式入門': 69, '程式_程式語言': 72, '程式_網頁前端': 74, '音樂_音樂創作': 147, '職場技能_資料彙整': 92, '職場技能_文書處理': 86, '職場技能_職場溝通': 91, '金融業': 135, '投資理財_理財': 27, '攝影_影像創作': 34, '藝術_更多藝術': 100, '音樂_樂器': 146, '投資理財_金融商品': 29, '資訊科技': 128, '法律': 47, '社會及文化專業': 62, '非營利組織': 142, '設計_設計理論': 119, '政治經濟': 39, '社會服務': 63, '舞台劇': 95, '電影': 138, '行銷_文案': 109, '攝影_商業攝影': 33, '插畫': 30, '桌遊': 44, '棋類遊戲': 45, '素描': 81, '電玩': 139, '藝文設計': 97, '自由業': 94, '音樂_人聲': 144, '程式_遊戲開發': 79, '藝術_字體設計': 98, '狗派': 49, '運動健身': 132, '電腦繪圖': 140, '出版業': 7, '職場技能_求職': 88, '職場技能_效率提升': 85, '設計_網頁設計': 118, '語言_更多語言': 121, '生活品味_親子教育': 58, '語言_英文': 124, '手作_手工印刷': 16, '手作_刺繡': 14, '手作_更多手作': 18, '手作_手工書': 17, '語言_韓文': 126, '手作_手作小物': 15, '寫作': 11, '手作': 13, '手寫字': 22, '手遊': 23, '水彩': 46, '速寫': 131, '閱讀': 137, '程式_程式思維': 70, '程式

# 推薦最相近的users，我拿items改的，input其實只會有一個user而已

In [22]:
def get_recommend_users(user_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(user2id_mapping))]
    user_index_list = [ user2id_mapping[user_id] for user_id in user_list]
    for idx in user_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the movies based on the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_indices = []
    # Get the scores of the 10 most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_indices) < top:
            if sim_scores[i][0] not in user_index_list:
                recommend_indices.append(sim_scores[i][0])

    # Return the top 10 most similar users
    return recommend_indices
    
recommend_indices = get_recommend_users(['54ccaa73a784960a00948687'], cosine_sim2, 5)
print(users_df['user_id'].iloc[recommend_indices].tolist())
print(f"原本user: {filledna['soup'].iloc[[user2id_mapping['54ccaa73a784960a00948687']]].tolist()}\n推薦:")
filledna['soup'].iloc[recommend_indices].tolist()


['57332668fad4ef0a006b22dc', '569e36cc4ec4c609007d2537', '57332b4cfad4ef0a006b236d', '59b9f021cd5792070055f6ef', '575c1b65f1399b0900e20d3d']
原本user: ['female  職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投資觀念,行銷_數位行銷,藝術_角色設計,藝術_繪畫與插畫,職場技能_個人品牌經營 ']
推薦:


['female  設計_介面設計,設計_動態設計,設計_網頁設計,藝術_電腦繪圖,藝術_繪畫與插畫,設計_平面設計,藝術_角色設計 ',
 'male  設計_介面設計,設計_動態設計,藝術_電腦繪圖,藝術_繪畫與插畫,設計_平面設計,藝術_角色設計 ',
 'female  投資理財_理財,設計_介面設計,設計_平面設計,設計_應用設計,藝術_電腦繪圖,設計_動態設計,投資理財_投資觀念,設計_網頁設計,藝術_角色設計,藝術_繪畫與插畫 ',
 'female  設計_平面設計,藝術_角色設計,藝術_電腦繪圖,設計_動態設計,行銷_數位行銷,藝術_繪畫與插畫 ',
 'male  藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_網頁設計,藝術_角色設計,藝術_繪畫與插畫 ']